# Speckle Stuff

In [ ]:
import datetime as dt

from dotenv import load_dotenv
from specklepy.api import operations
from specklepy.api.client import SpeckleClient
from specklepy.api.credentials import get_default_account
from specklepy.objects import Base
from specklepy.objects.geometry import Line, Point
from specklepy.transports.server import ServerTransport


def to_speckle():
    load_dotenv()
    stream_id = "7fbe8ed384"
    hole_table_path = "data/1_split2/HOLE.csv"
    df = pd.read_csv(hole_table_path, index_col=0)
    client, stream_id = get_stream(stream_id)

    # next create a server transport - this is the vehicle through which you will send and receive
    transport = ServerTransport(client=client, stream_id=stream_id)

    hash = create_hash(df, transport)

    commit_hash(hash, client, stream_id)


def get_stream(stream_id):
    # Authenticate with Speckle server
    speckle_server = "app.speckle.systems"
    speckle_token = os.environ["speckle_token"]
    client = SpeckleClient(host=speckle_server)
    account = get_default_account()

    client.authenticate_with_token(speckle_token)

    # create a new stream. this returns the stream id
    if not stream_id:
        stream_id = client.stream.create(name="a shiny new stream")

    # use that stream id to get the stream from the server
    new_stream = client.stream.get(id=stream_id)
    return client, stream_id


def create_hash(df, transport):
    newObj = Base()
    for i, row in df.iterrows():
        x = row["HOLE_NATE"]
        y = row["HOLE_NATN"]
        z_top = row["HOLE_GL"]
        z_bot = z_top - row["HOLE_FDEP"]

        # GisPointElement, GisLineElement lijken niet te werken
        p1 = Point(x=x, y=y, z=z_top)
        p2 = Point(x=x, y=y, z=z_bot)
        line = Line(start=p1, end=p2)
        relevant_cols = [
            "HOLE_ID",
            "HOLE_TYPE",
            "HOLE_STAR",
            "HOLE_LOG",
            "?HOLE_DLOG",
            "?HOLE_CHEK",
            "?HOLE_DCHK",
            "HOLE_REM",
            "?HOLE_FLSH",
            "HOLE_ENDD",
            "HOLE_BACD",
            "HOLE_CREW",
            "HOLE_INCL",
            "HOLE_EXC",
        ]

        for col in relevant_cols:
            line[col] = row[col]

        newObj[f"myline{i}"] = line
        break

    # this serialises the block and sends it to the transport
    hash = operations.send(base=newObj, transports=[transport])
    return hash


def commit_hash(hash, client, stream_id):
    # you can now create a commit on your stream with this object
    commid_id = client.commit.create(
        stream_id=stream_id,
        object_id=hash,
        message=f"these are lines I made in speckle-py at {dt.datetime.now()}",
    )


if __name__ == "__main__":
    to_speckle()
